# Example 1: Query a table with SQL and magic commands

In [ ]:
# Running this code will query a table in BigQuery and download
# the results to a Pandas DataFrame named `results`.
# Learn more here: https://cloud.google.com/bigquery/docs/visualize-jupyter

%%bigquery results --project jpmc2-468719
SELECT * FROM `jpmc2-468719.Delaware_Checkbook_Expenditure.Expenditure` #this table name was set based on the table you chose to query

In [ ]:
# You can view the resulting Pandas DataFrame and work with using the Pandas library.
# https://pandas.pydata.org/docs/getting_started/index.html#getting-started
results

# Example 2: Query a table with BigQuery DataFrames

In [ ]:
# With BigQuery DataFrames, you can use many familiar Pandas methods, but the
# processing happens in BigQuery rather than the runtime, allowing you to work with larger
# DataFrames that would otherwise not fit in the runtime memory.
# Learn more here: https://cloud.google.com/python/docs/reference/bigframes/latest

import bigframes.pandas as bf

bf.options.bigquery.location = "us-east1" #this variable is set based on the dataset you chose to query
bf.options.bigquery.project = "jpmc2-468719" #this variable is set based on the dataset you chose to query

In [ ]:
df = bf.read_gbq("jpmc2-468719.Delaware_Checkbook_Expenditure.Expenditure") #this variable is set based on the dataset you chose to query

In [ ]:
# BigFrames can work with tables that are too large to fit in the notebook memory.
# Look at the first 20 rows.
df.head(20)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')
#  DATA LOADING & PREPROCESSING
def load_and_preprocess_data_fast(file_path, sample_size=None):
    """
    Fast data loading with optional sampling for large datasets
    """
    print("Loading dataset...")
    # Load with efficient dtypes
    df = pd.read_csv(file_path, dtype={
        'Department': 'category',
        'Vendor': 'category',
        'Category': 'category'
    })
    # Sample if dataset is too large
    if sample_size and len(df) > sample_size:
        print(f" Sampling {sample_size:,} rows from {len(df):,} total rows")
        df = df.sample(n=sample_size, random_state=42)
    # Auto-detect column mappings
    column_mapping = {}
    for col in df.columns:
        col_lower = col.lower()
        if 'department' in col_lower or 'agency' in col_lower:
            column_mapping[col] = 'department'
        elif 'vendor' in col_lower or 'supplier' in col_lower:
            column_mapping[col] = 'vendor'
        elif 'amount' in col_lower or 'total' in col_lower or 'payment' in col_lower:
            column_mapping[col] = 'amount'
        elif 'category' in col_lower or 'type' in col_lower:
            column_mapping[col] = 'category'
        elif 'description' in col_lower or 'item' in col_lower:
            column_mapping[col] = 'description'
    df = df.rename(columns=column_mapping)
    # Clean data efficiently
    if 'amount' in df.columns:
        df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
        df = df.dropna(subset=['amount'])
        df = df[df['amount'] > 0]
    # Clean text columns
    text_cols = ['department', 'vendor', 'category', 'description']
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.upper()
    print(f" Loaded {len(df):,} records")
    print(f" Columns: {list(df.columns)}")
    print(f" Total spending: ${df['amount'].sum():,.2f}")
    return df
#  PRICE VARIANCE ANALYSIS
def fast_price_variance_analysis(df, max_categories=20):
    """
    Optimized price variance analysis without fuzzy matching
    """
    print(" Starting fast price variance analysis...")
    results = []
    # Focus on categories with sufficient data
    if 'category' in df.columns:
        category_counts = df['category'].value_counts()
        top_categories = category_counts.head(max_categories).index
        for category in top_categories:
            category_data = df[df['category'] == category]
            if len(category_data) < 5:  # Skip categories with too few records
                continue
# Use vectorized operations
            price_stats = {
                'item_name': f"Category: {category}",
                'num_purchases': len(category_data),
                'mean_price': category_data['amount'].mean(),
                'median_price': category_data['amount'].median(),
                'std_price': category_data['amount'].std(),
                'min_price': category_data['amount'].min(),
                'max_price': category_data['amount'].max(),
                'total_spent': category_data['amount'].sum()
            }
            # Calculate variance metrics
            price_stats['coefficient_of_variation'] = (
                price_stats['std_price'] / price_stats['mean_price']
                if price_stats['mean_price'] > 0 else 0
            )
            # Potential savings calculation
            price_stats['potential_savings'] = (
                price_stats['total_spent'] -
                (price_stats['num_purchases'] * price_stats['min_price'])
            )
            price_stats['savings_percentage'] = (
                price_stats['potential_savings'] / price_stats['total_spent'] * 100
                if price_stats['total_spent'] > 0 else 0
            )
            # Add department/vendor info
            if 'department' in df.columns:
                price_stats['num_departments'] = category_data['department'].nunique()
            if 'vendor' in df.columns:
                price_stats['num_vendors'] = category_data['vendor'].nunique()
            # Priority classification
            if price_stats['potential_savings'] > 50000:
                price_stats['priority'] = 'CRITICAL'
            elif price_stats['potential_savings'] > 10000:
                price_stats['priority'] = 'HIGH'
            elif price_stats['potential_savings'] > 5000:
                price_stats['priority'] = 'MEDIUM'
            else:
                price_stats['priority'] = 'LOW'
            # Only include items with significant variation
            if price_stats['coefficient_of_variation'] > 0.2:
                results.append(price_stats)
# Department-Vendor analysis (vectorized)
    if 'department' in df.columns and 'vendor' in df.columns:
        dept_vendor_stats = df.groupby(['department', 'vendor']).agg({
            'amount': ['count', 'mean', 'std', 'sum']
        }).reset_index()
        # Flatten column names
        dept_vendor_stats.columns = [
            'department', 'vendor', 'count', 'mean_amount', 'std_amount', 'total_amount'
        ]
        # Find high-variation department-vendor pairs
        dept_vendor_stats['cv'] = dept_vendor_stats['std_amount'] / dept_vendor_stats['mean_amount']
        high_variation = dept_vendor_stats[
            (dept_vendor_stats['cv'] > 0.3) &
            (dept_vendor_stats['count'] > 3) &
            (dept_vendor_stats['total_amount'] > 5000)
        ]
        for _, row in high_variation.iterrows():
            results.append({
                'item_name': f"Dept: {row['department']} | Vendor: {row['vendor']}",
                'num_purchases': row['count'],
                'mean_price': row['mean_amount'],
                'std_price': row['std_amount'],
                'coefficient_of_variation': row['cv'],
                'total_spent': row['total_amount'],
                'potential_savings': row['total_amount'] * 0.1,  # Estimate 10% savings
                'priority': 'HIGH' if row['total_amount'] > 25000 else 'MEDIUM'
            })
    return sorted(results, key=lambda x: x.get('potential_savings', 0), reverse=True)
def fast_clustering_analysis(df, sample_size=10000):
    """
    Fast clustering with sampling for large datasets
    """
    print(" Starting fast clustering analysis...")
    # Sample for clustering if dataset is large
    if len(df) > sample_size:
        print(f"Sampling {sample_size:,} rows for clustering")
        cluster_df = df.sample(n=sample_size, random_state=42)
    else:
        cluster_df = df.copy()
    # Prepare features efficiently
    features = pd.DataFrame()
    features['amount'] = cluster_df['amount']
    features['log_amount'] = np.log1p(cluster_df['amount'])
    # Encode categoricals efficiently
    le_dict = {}
    for col in ['department', 'vendor', 'category']:
        if col in cluster_df.columns:
            le = LabelEncoder()
            features[f'{col}_encoded'] = le.fit_transform(cluster_df[col].astype(str))
            le_dict[col] = le
    # Add frequency features
    if 'vendor' in cluster_df.columns:
        vendor_counts = cluster_df['vendor'].value_counts()
        features['vendor_frequency'] = cluster_df['vendor'].map(vendor_counts)
    # Scale features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features.fillna(0))
    # Fast K-means with fewer clusters
    n_clusters = min(8, len(features) // 100)  # Adaptive cluster count
    if n_clusters >= 2:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=5)
        cluster_labels = kmeans.fit_predict(features_scaled)
        # Analyze clusters
        cluster_df['cluster'] = cluster_labels
        cluster_summary = []
        for cluster_id in range(n_clusters):
            cluster_data = cluster_df[cluster_df['cluster'] == cluster_id]
            summary = {
                'cluster_id': cluster_id,
                'size': len(cluster_data),
                'percentage': len(cluster_data) / len(cluster_df) * 100,
                'avg_amount': cluster_data['amount'].mean(),
                'median_amount': cluster_data['amount'].median(),
                'total_spending': cluster_data['amount'].sum()
            }
            if 'department' in cluster_data.columns:
                summary['top_department'] = cluster_data['department'].mode().iloc[0]
                summary['num_departments'] = cluster_data['department'].nunique()
            cluster_summary.append(summary)
        return {
            'cluster_labels': cluster_labels,
            'n_clusters': n_clusters,
            'cluster_analysis': cluster_summary,
            'features_used': list(features.columns)
        }
    return None
# FAST VISUALIZATION
def create_fast_visualizations(price_results, clustering_results=None):
    """
    Create quick visualizations of results
    """
    print(" Creating visualizations...")
    if price_results:
        # Top opportunities chart
        top_10 = price_results[:10]
        if top_10:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            # Savings potential
            items = [r['item_name'][:30] + '...' if len(r['item_name']) > 30
                    else r['item_name'] for r in top_10]
            savings = [r.get('potential_savings', 0) for r in top_10]
            ax1.barh(items, savings)
            ax1.set_xlabel('Potential Savings ($)')
            ax1.set_title('Top 10 Consolidation Opportunities')
            ax1.tick_params(axis='y', labelsize=8)
            # Price variation
            cv_values = [r.get('coefficient_of_variation', 0) * 100 for r in top_10]
            ax2.barh(items, cv_values)
            ax2.set_xlabel('Price Variation (%)')
            ax2.set_title('Price Variation by Category')
            ax2.tick_params(axis='y', labelsize=8)
            plt.tight_layout()
            plt.show()
    if clustering_results:
        # Cluster distribution
        cluster_analysis = clustering_results.get('cluster_analysis', [])
        if cluster_analysis:
            sizes = [c['size'] for c in cluster_analysis]
            labels = [f"Cluster {c['cluster_id']}" for c in cluster_analysis]
            plt.figure(figsize=(10, 6))
            plt.pie(sizes, labels=labels, autopct='%1.1f%%')
            plt.title('Spending Pattern Clusters')
            plt.show()
#  FAST ANALYSIS FUNCTION
def run_fast_analysis(file_path, sample_size=50000):
    """
    Run optimized analysis for Colab
    """
    print("DELAWARE EXPENDITURE ANALYSIS")
    print("=" * 50)
    # Load data with sampling
    df = load_and_preprocess_data_fast(file_path, sample_size=sample_size)
    # Fast price analysis
    print("\n Running price variance analysis...")
    price_results = fast_price_variance_analysis(df, max_categories=15)
    print(f"\n Found {len(price_results)} consolidation opportunities")
    if price_results:
        total_savings = sum(r.get('potential_savings', 0) for r in price_results)
        print(f":moneybag: Total potential savings: ${total_savings:,.2f}")
        print("\n TOP 5 OPPORTUNITIES:")
        for i, result in enumerate(price_results[:5], 1):
            print(f"{i}. {result['item_name'][:50]}")
            print(f"    Savings: ${result.get('potential_savings', 0):,.2f}")
            print(f"    Variation: {result.get('coefficient_of_variation', 0):.1%}")
    # Fast clustering
    print("\n Running clustering analysis...")
    clustering_results = fast_clustering_analysis(df, sample_size=10000)
    if clustering_results:
        print(f" Identified {clustering_results['n_clusters']} spending patterns")
    # Quick visualizations
    create_fast_visualizations(price_results, clustering_results)
    return {
        'price_results': price_results,
        'clustering_results': clustering_results,
        'dataset_info': {
            'total_records': len(df),
            'total_spending': df['amount'].sum(),
            'departments': df['department'].nunique() if 'department' in df.columns else 0,
            'vendors': df['vendor'].nunique() if 'vendor' in df.columns else 0
        }
    }





if __name__ == "__main__":
    # Example usage in Colab
    FILE_PATH = "Delaware_Checkbook_Expenditure_Details_20250813.csv"  # Your file path
    # Run fast analysis (samples large datasets automatically)
    results = run_fast_analysis(FILE_PATH, sample_size=50000)
    print("\n Analysis complete!")
    print(" Key optimizations applied:")
    print("   • Removed slow fuzzy matching")
    print("   • Used vectorized pandas operations")
    print("   • Sampled large datasets")
    print("   • Focused on top categories only")
    print("   • Efficient clustering with fewer iterations")


In [ ]:
# Load data from BigQuery.
table = spark.read.format('bigquery') \
  .option('table', 'jpmc2-468719.Delaware_Checkbook_Expenditure.Expenditure') \
  .load()
table.createOrReplaceTempView('table')

# Explore the data and schema.
table_data = spark.sql(
    'SELECT * FROM penguins')
table_data.show()
table_data.printSchema()